# LSTM for classifying feelings (IMDb dataset)

### Import stuff

In [45]:
import io
import os
import re
import shutil
import string
import tqdm
# import kormos
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, LSTM, Flatten
from keras.layers import TextVectorization
from aux_we import generate_training_data
%load_ext tensorboard
%reload_ext tensorboard
# mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1", "/gpu:2", "/gpu:3", "/gpu:4"])
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('CPU')))

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


#### Dataset

Downloading dataset

In [46]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

if not(os.path.exists('aclImdb_v1.tar.gz')):
    print("===== Downloading Imdb Dataset =====")
    dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

    dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
    train_dir = os.path.join(dataset_dir, 'train')
    remove_dir = os.path.join(train_dir, 'unsup')
    shutil.rmtree(remove_dir)

Processing downloaded dataset

In [47]:
batch_size = 1024
seed = 127
train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)
val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)

#Change to true to see a sample
print_one = False
if print_one:
    for text_batch, label_batch in train_ds.take(1):
        for i in range(1):
            print(f"Review: {text_batch.numpy()[i]}")
            print(f"Label: {label_batch.numpy()[i]}")
            print()

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [48]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Standardizing the data

In [49]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  nots = tf.strings.regex_replace(lowercase, 'n\'t', ' not')
  ss = tf.strings.regex_replace(nots, '\'s', '')
  stripped_html = tf.strings.regex_replace(ss, '<br />', ' ')
  no_ponctuation = tf.strings.regex_replace(stripped_html,'[%s]' % re.escape(string.punctuation), '')
  single_spaces = tf.strings.regex_replace(no_ponctuation, '  ', ' ')
  for i in range(2):
    single_spaces = tf.strings.regex_replace(single_spaces, '  ', ' ')
  return single_spaces

dictionary_size = 2000
max_review_size = 250
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=dictionary_size,
    output_mode='int',
    output_sequence_length=max_review_size)

#Build dictonary
text_ds = train_ds.map(lambda x, y: x)
text_ds = text_ds.cache().prefetch(buffer_size=10)
vectorize_layer.adapt(text_ds)

#Print one
# text_batch, label_batch = next(iter(train_ds))
# first_review, first_label = text_batch[10], label_batch[10]
# print("Review", custom_standardization(first_review))
# print("Label", train_ds.class_names[first_label])
# print("Vectorized review", vectorize_layer(first_review))

In [44]:
print_one = True
if print_one:
    text_batch, label_batch = next(iter(train_ds))
    first_review, first_label = text_batch[10], label_batch[10]
    print("Review", custom_standardization(first_review))
    print("Label", first_label)
    print("Vectorized review", vectorize_layer(first_review))

Review tf.Tensor(b'its no surprise that busey later developed a tumor in his sinus cavity this film is also a poor decision but one i enjoyed fully the first 5 minutes is the most uninspiring 5 minutes in any film boring bad dialouge and then with a spiderman stance busey yells the bestworst line in any film ever createdyour worst nightmare butthorn i coughed up some of my egg nog laughing so hard that line resonates so well it even tops clooney infamous hi freeze im batman line other classic moments is busey constantly getting upset for people reminding him that he got his excia partner killedwhich he did by accidentally shooting him in the chest all made possible by a super slowmotion flashback sequence that makes watching paint dry seem exciting there an ashtray to the nads punches to the face and a that was not my fault and you know it well the footage shows him missing the bad guy and hitting his buddy so other scream outloud moments has to be his ex girlfriend dropping a grenade 

## Word Embedding

Dataset for training the word embedding

In [50]:
print_stuff = False

inverse_vocab = vectorize_layer.get_vocabulary()
if print_stuff:
    print('Part of vocab:',inverse_vocab[:200])

# Vectorize the data in text_ds
text_vector_ds = text_ds.prefetch(AUTOTUNE).map(vectorize_layer)
lst = list(text_vector_ds.as_numpy_iterator())
sequences = lst[0]
for i in range(1,len(lst)):
    arr = np.asarray(lst[i])
    sequences = np.append(sequences, arr, 0)
sequences.reshape(-1)
print(sequences.shape)

if print_stuff:
    print(len(sequences)*batch_size)
    for seq in sequences[2][:1]:
        print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

# Create
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=5,
    neg_samples=10,
    vocab_size=dictionary_size,
    seed=seed)

targets = np.array(targets)
contexts = np.array(contexts)[:,:,0]
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")


(20000, 250)


 33%|███▎      | 6525/20000 [03:31<07:17, 30.83it/s]


KeyboardInterrupt: 

Optimize dataset

In [29]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

2022-10-12 17:09:20.032093: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3188231448 exceeds 10% of free system memory.
2022-10-12 17:09:22.395955: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3188231448 exceeds 10% of free system memory.
2022-10-12 17:09:24.589316: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3188231448 exceeds 10% of free system memory.
2022-10-12 17:09:25.383201: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3188231448 exceeds 10% of free system memory.


<BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 101), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 101), dtype=tf.int64, name=None))>


Train embedding

In [31]:
embedding_dim=10
neg_samples=20

#Train WE
class Word2Vec(tf.keras.Model):
  def __init__(self, dictionary_size=dictionary_size, embedding_dim=embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(dictionary_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(dictionary_size,
                                       embedding_dim,
                                       input_length=neg_samples+1)
  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

w2v = Word2Vec(dictionary_size, embedding_dim)

w2v.compile(optimizer='adam',
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])
            
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
w2v.fit(dataset,
       epochs=200)


Epoch 1/200
3853/3853 [==============================] - 17s 4ms/step - loss: 4.3908 - accuracy: 0.0639
Epoch 2/200
3853/3853 [==============================] - 17s 4ms/step - loss: 4.3158 - accuracy: 0.0730
Epoch 3/200
 836/3853 [=====>........................] - ETA: 12s - loss: 4.2941 - accuracy: 0.0753

KeyboardInterrupt: 

Save weigts of trained embedding

In [ ]:
weights = w2v.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

Save vectors and words in .tsv

In [ ]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

Process vectors from tsv

In [ ]:
import pandas as pd

df = pd.read_csv('vectors.tsv', sep="\t")
df2 = pd.read_csv('metadata.tsv', sep="\t")

vecs = df.values
wrds = df2.values

num_tokens = dictionary_size

#Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for i in range(1,len(vecs)+1):
    embedding_matrix[i] = vecs[i]
    if (i%100==0):
        print(wrds[i], '=', inverse_vocab[i])

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

os.remove('metadata.tsv')
os.remove('vectors.tsv')

## LSTM itself

In [ ]:
model = Sequential([
  vectorize_layer,
  embedding_layer,
  LSTM(4),
  Dense(2),
  Dense(1, activation='sigmoid')
])

os.rmdir("logs")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[tensorboard_callback])

#docs_infra: no_execute
%tensorboard --logdir logs

Epoch 1/15
20/20 [==============================] - 3s 126ms/step - loss: 0.6935 - accuracy: 0.4995 - val_loss: 0.6930 - val_accuracy: 0.5020
Epoch 2/15
20/20 [==============================] - 3s 142ms/step - loss: 0.6930 - accuracy: 0.4995 - val_loss: 0.6928 - val_accuracy: 0.5020
Epoch 3/15
20/20 [==============================] - 2s 116ms/step - loss: 0.6928 - accuracy: 0.4995 - val_loss: 0.6926 - val_accuracy: 0.5020
Epoch 4/15
20/20 [==============================] - 2s 93ms/step - loss: 0.6926 - accuracy: 0.4995 - val_loss: 0.6925 - val_accuracy: 0.5020
Epoch 5/15
20/20 [==============================] - 2s 91ms/step - loss: 0.6924 - accuracy: 0.4995 - val_loss: 0.6924 - val_accuracy: 0.5020
Epoch 6/15
20/20 [==============================] - 2s 93ms/step - loss: 0.6923 - accuracy: 0.4995 - val_loss: 0.6923 - val_accuracy: 0.5020
Epoch 7/15
20/20 [==============================] - 2s 106ms/step - loss: 0.6922 - accuracy: 0.4995 - val_loss: 0.6922 - val_accuracy: 0.5020
Epoch 8/1

Reusing TensorBoard on port 6006 (pid 69154), started 2 days, 14:04:49 ago. (Use '!kill 69154' to kill it.)